#### CIE4604 Simulation and Visualization

# Module 3 2.5D Visualization - Exercise 2 (Relief, Contour and Surface plots)

**Hans van der Marel, 26 November 2021**

In this exercise you are going to explore several other options to do 2.5D visualizations in Python, these include adding relief, making contour maps and 3D surface plots. The data which we will be using is a *crop* of the 100 meter resolution data from the *Actueel Hoogtebestand Nederland* (AHN-1), integrated with Bathemetric data, to cover the rivers, lakes and the Dutch coast.

## Import the necesary modules

For the visualization we use again `matplotlib.pyplot`, the NetCDF `netCDF4` module to read the netCDF file with DEM data, and `numpy` for the general math.

In [1]:
# Importing the libraries
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

## Read a data crop from netCDF file *mv100.nc* 

The elevation dataset is given in an NetCDF file `mv100.nc` which was used also in exercise 1. See exercise 1 to learn more about exploring and reading variables from a netCDF file.

For this exercise we will be reading a subset, or crop, from the dataset. With netCDF this can be done without loading the full dataset in memory, suing the following code.

In [ ]:
# Create the grid object for the data
grid = nc.Dataset('mv100.nc')
# Retrieve a subset (crop) of the data
xx = grid.variables['x'][1000:1500]
yy = grid.variables['y'][500:1000]
zz = grid.variables['depth'][1000:1500, 500:1000].T

Notice that we transposed the height/depth data matrix *zz* directly while reading in order to get the axis alignment right. We can do a quick check with the `matplot` function

In [ ]:
plt.matshow(zz)

## Select colormap and color axis limits for pseudo-color plots

In exercise 1 we experimented with the colormap and limits for the color axis. In this exercise we use the colormap that we decided upon in exercise 1, and adjust it to the z-data range in the subset. In the next code sections we

1. make a histogram of the z-data,
2. select the colormap and c-axis limits, and assign them to variables *cmap* and *clim* that will used throughtout,
3. make a test plot, using `imshow` with our selection. 

We are also going to use the `extent=[x_min, x_max, y_min, y_max]` of `imshow` to get the axis ticks right in `imshow`. In this way  we can use `imshow` instead of `pcolormesh`. This is only possible for regular grids which use the same increments for the x- and/or y-axis.  

In [ ]:
# histogram of the data
hist, bin_edges = np.histogram(zz, np.arange(-40,20,1), range=(-40, 20), density=False)

# plot the histogram
plt.figure(figsize=(10,5), tight_layout=True)
plt.bar(bin_edges[0:-1], hist, align='edge')
plt.xlabel('Height/depth [m]')
plt.ylabel('Count [-]')
plt.title('Histogram height/depth values in the crop')
plt.show()

# plot the histogram
#plt.figure(figsize=(10,5), tight_layout=True)
#plt.hist(np.array(zz).flat, np.arange(-40,20,1), range=(-40, 20), density=False)
#plt.xlabel('Height/depth [m]')
#plt.ylabel('Count [-]')
#plt.title('Histogram height/depth values in the crop')
#plt.show()

In [ ]:
# Select the colormap to use and c-axis limits 
#cmap = plt.cm.gist_earth    # colormap
cmap = plt.cm.terrain        # colormap
clim = (-10, 30)             # clim

# Get the extend of the x- and y-range,so that we can set extent in imshow to display the correct x- and y-ticks
extent = [np.min(xx)/1000 , np.max(xx)/1000, np.min(yy)/1000 , np.max(yy)/1000]

The colormap is created as an object. The object *cmap* is a callable, when passed a float between 0 and 1, returns an RGBA (Red, Green, Blue, Alpha) value from the colormap.

In [ ]:
print(cmap)
print(cmap(0.56))
print(cmap(np.arange(0,1,.2)))

The colormap is either class `ListedColormap` or `LinearSegmentedColormap`. Both map values between 0 and 1 to a bunch of colors. 

A `ListedColormap` is a lookup table which returns nearest-neighbor interpolated values. The list of colors that comprise the colormap can be directly accessed using the colors property, which returns a list is in the form of an RGBA Nx4 array, where N is the length of the colormap. To create a new listed colormap supply a list or array of color specifications to ListedColormap,
```Python
   cmap = ListedColormap(["darkorange", "gold", "lawngreen", "lightseagreen"])
```
Nearest-neighbor interpolation is used in the lookup, so this colormap give at most four colors when used. More useful for creating custom colormaps is to used Nx4 numpy arrays. In the following example we craft our own colormap
```Python
   N = 256
   vals = np.ones((N, 4))
   vals[:, 0] = np.linspace(90/256, 1, N)
   vals[:, 1] = np.linspace(40/256, 1, N)
   vals[:, 2] = np.linspace(40/256, 1, N)
   newcmp = ListedColormap(vals)
```
An other example is to merge two existing colormaps
```Python
   top = cm.get_cmap('Oranges_r', 128)
   bottom = cm.get_cmap('Blues', 128)
   newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
   newcmp = ListedColormap(newcolors, name='OrangeBlue')
```

The `LinearSegmentedColormap` class specifies colormaps using anchor points between which RGB(A) values are interpolated.
A such, linear segmented colormaps do not have a .colors attribute.
Each anchor point is specified as a row in a matrix of the form *\[x\[i\] yleft\[i\] yright\[i\]\]*, where *x\[i\]* is the anchor, and *yleft\[i\]* and *yright\[i\]* are the values of the color on either side of the anchor point. This format allows discontinuities at the anchor points. If there are no discontinuities, then *yleft\[i\]=yright\[i\]*. 
An example is
```Python
cdict = {'red':   [[0.0,  0.0, 0.0],
                   [0.5,  1.0, 1.0],
                   [1.0,  1.0, 1.0]],
         'green': [[0.0,  0.0, 0.0],
                   [0.25, 0.0, 0.0],
                   [0.75, 1.0, 1.0],
                   [1.0,  1.0, 1.0]],
         'blue':  [[0.0,  0.0, 0.0],
                   [0.5,  0.0, 0.0],
                   [1.0,  1.0, 1.0]]}
newcmp = LinearSegmentedColormap('testCmap', segmentdata=cdict, N=256)
```
For some basic cases, the use of `LinearSegmentedColormap.from_list` may be easier. This creates a segmented colormap with equal spacings from a supplied list of colors,
```Python
   colors = ["darkorange", "gold", "lawngreen", "lightseagreen"]
   cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
```
If you don't want equal spacings, the nodes of the colormap can be given as numbers between 0 and 1. E.g. one could have the reddish part take more space in the colormap by specifying nodes as
```Python
   nodes = [0.0, 0.4, 0.8, 1.0]
   cmap2 = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors)))
```

In [ ]:
from matplotlib import colors

# make a colormap that has land and water clearly delineated and of the
# same length (256 + 256)
colors_water = plt.cm.terrain(np.linspace(0, 0.17, 256))
colors_land = plt.cm.terrain(np.linspace(0.17, 1, 256))
all_colors = np.vstack((colors_water, colors_land))
terrain_map = colors.LinearSegmentedColormap.from_list(
    'terrain_map', all_colors)

# make the norm:  Note the center is offset so that the land has more
# dynamic range:
divnorm = colors.TwoSlopeNorm(vmin=-10., vcenter=-3, vmax=20)

# Plot using imshow, with the proper extent 
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plt.imshow(zz, cmap=terrain_map, norm=divnorm, aspect='equal', extent=extent)
plt.colorbar(label='NAP Height [m]')
plt.xlabel('X-RD [km]')
plt.ylabel('Y-RD [km]')
plt.title('Digital Elevation Model for the NW Nederlands')
plt.show()

In [ ]:
# Plot using imshow, with the proper extent 
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plt.imshow(zz, cmap=cmap, clim=clim, aspect='equal', extent=extent)
plt.colorbar(label='NAP Height [m]')
plt.xlabel('X-RD [km]')
plt.ylabel('Y-RD [km]')
plt.title('Digital Elevation Model for the NW Nederlands')
plt.show()

The crop shows the island Texel, the western part of the Dutch Waddenzee, the north of the province Noord-Holland, Wieringer meer polder, and a section of the Ijsselmeer lake. This is a low area in the Netherlands, with polders below sea-level and some dunes, and a lot of water areas with some deep active tidal channels, and remains of old tidal channels in the Ijsselmeer lake. 

The Afsluitdijk, a dike separating the Waddenzee from the Ijselmeer lake, is clearly visible, as well as the *Texel stroom*, a big tidal inlet running between the province of Noord-Holland and the island of Texel. On land, the dunes along the coast are easily distinquishable, as well as the high areas of the former island of Wieringen. But, although you can see the dike between the polder of Wieringermeer and the Ijsselmeer quite well, the Wieringermeer polder itself is hardly distinquishable from the Ijsselmeer to which it formerly belonged.

Although there is already a lot to see, the pseudo colour image itself look quite "flat", despite all the efforts we put in finding a good colour map. Time to light things up and add some shading to reveal more details.


## Shaded relief plots

In the next cells we are going to make some shaded relief plots by adding a light source and computing the illumination intensity depending on gradients in the height/depth data.

First import `LightSource` from `matplotlob.colors`, then define a light source *ls* by giving the azimuth and elevation of the source, and then show the illumination intensity and two different ways of blending the pseudo colors with the illumination intensity to obtain the desired effect. We also defined a variable *ve* to set the vertical exageration. 

In [ ]:
from matplotlib.colors import LightSource

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(12,12), tight_layout=True)
for ax in axs.flat:
    ax.set(xticks=[], yticks=[])

# Illuminate the scene from the northwest
ls = LightSource(azdeg=315, altdeg=45)
ve = 0.5   # Vertical exageration

axs[0, 0].imshow(zz, cmap=cmap, clim= clim)
axs[0, 0].set(xlabel='Colormapped Data')

axs[0, 1].imshow(ls.hillshade(zz, vert_exag=ve), cmap='gray')
axs[0, 1].set(xlabel='Illumination Intensity')

rgb = ls.shade(zz, cmap=cmap, vmin= clim[0], vmax=clim[1], vert_exag=ve, blend_mode='hsv')
axs[1, 0].imshow(rgb)
axs[1, 0].set(xlabel='Blend Mode: "hsv" (default)')

rgb = ls.shade(zz, cmap=cmap, vmin= clim[0], vmax=clim[1], vert_exag=ve, blend_mode='overlay')
axs[1, 1].imshow(rgb)
axs[1, 1].set(xlabel='Blend Mode: "overlay"')

plt.show()

Overlay blending seems to give visually the best result, but the hsv blending shows (exagerates) more details. 

The code below shows the final result for the overlay mode

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8))

# Illuminate the scene from the northwest
ls = LightSource(azdeg=315, altdeg=45)
ve = 0.5   # Vertical exageration

rgb = ls.shade(zz, cmap=cmap, vmin= clim[0], vmax=clim[1], vert_exag=ve, blend_mode='overlay')
ax.imshow(rgb, aspect='equal', extent=extent)
norm = plt.Normalize(vmin=clim[0], vmax=clim[1], clip=False)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
fig.colorbar(sm,label='NAP Height [m]')
ax.set(xlabel= 'X-RD [km]', ylabel='Y-RD [km]', title='Digital Elevation Model for the NW Nederlands')
plt.show()

## Plotting contour lines

Another option is to add contour lines to the plots. Below we show two examples.

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
levels = [ -10, -5, 0, 5, 10 ]

ax.imshow(rgb, aspect='equal', extent=extent)
fig.colorbar(sm,label='NAP Height [m]')

#ax.contourf(xx/1000, yy/1000, zz, levels=80, cmap=cmap, vmin=clim[0], vmax=clim[1])
ax.contour(xx/1000, yy/1000, zz, levels=levels, colors='gray', vmin=clim[0], vmax=clim[1], linewidths=1)
plt.show()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
levels1= [-40, -20, -10, -7, -5, -3, -2, -1, -0.5, 0, 0.5, 1, 2 , 3, 5, 10, 20]
levels2 = [ -10, -5, 0 ]
#cset = ax.contourf(xx/1000, yy/1000, zz, cmap=cmap, vmin=clim[0], vmax=clim[1])
ax.contourf(xx/1000, yy/1000, zz, levels=levels1, cmap=cmap, vmin=clim[0], vmax=clim[1])
ax.contour(xx/1000, yy/1000, zz, levels=levels2, colors='k', vmin=clim[0], vmax=clim[1], linewidths=1)
ax.axis('equal')
ax.axis('tight')
plt.show()

## 3D contour, wireframe and surface plots

3D plots are possible in `matplotlib` when `projections=3d` is selected for the axis and the `Axis3D` module is imported from `mpl_toolkits.mplot3d`.

Several types of plots are possible. We show a few examples without much explanation.

> 3D plots are best displayed with `Qt` or using `notebook` instead of `inline` plots. Use either
> ```
> %matplotlib qt
> %matplotlib notebook
> ```
>
> Using the mouse you can zoom, pan and rotate the plots

To return to inline plots specify `%matplotlob inline`

### 3D contour plots

Contours can also be plotted in 3D by using the `projections=3d` on the axis. We show two results with different number of levels.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# 3D plot (better visualised in an IDE rather than Notebook)
%matplotlib qt
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
cset = ax.contourf(xx/1000, yy/1000, zz, cmap='terrain', levels=levels1, vmin=clim[0], vmax=clim[1])
plt.show()

In [ ]:
# 3D plot (better visualised in an IDE rather than Notebook)
%matplotlib qt
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
cset = ax.contourf3D(xx/1000, yy/1000, zz, cmap='terrain', levels=range(clim[0],clim[1],1), vmin=clim[0], vmax=clim[1])
plt.show()

### 3D wireframe

Another option is to plot a wireframe. This cannot be done for the individual data points (there are too many), so we selected a stride of 20 in each direction.

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(xx, yy)
#cset = ax.plot_surface(X/1000, Y/1000, zz, linewidth=0, rstride=20, cstride=20) #cmap=cmap)
cset = ax.plot_wireframe(X/1000, Y/1000, zz, rstride=20, cstride=20) #cmap=cmap)
#cset = ax.contour3D(X/1000, Y/1000, zz, cmap=cmap)

plt.show()

### 3D surface plots

If we select a subset of the data then it becomes possible to do true 3D surface plot. Here is an example, with contours plotted in the xy plane.

In [ ]:
xxx=xx[30:130]
yyy=yy[200:300]
zzz=zz[30:130,200:300]

%matplotlib qt
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(xxx, yyy)
csurf = ax.plot_surface(X/1000, Y/1000, zzz, cmap=cmap, clim=clim)
cset = ax.contourf(X/1000, Y/1000, zzz, zdir='z', offset=np.min(zzz), cmap=cmap, vmin=clim[0], vmax=clim[1])
#cset = ax.plot_wireframe(X/1000, Y/1000, zzz)
cset = ax.contour3D(X/1000, Y/1000, zzz, cmap=cmap, vmin=clim[0], vmax=clim[1])

plt.show()

[End of notebook]